# Modeling for predicting the condo size
## linear regression
in this part, we are going to use linear regression to predict the price per square of the house. Compared to the model above, we are going to use another representation of the data.

In [25]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('out.csv',header=0,index_col=0)

# inorder to avoid Multicollinearity, we have to delete several columns.
df.drop(columns = ['E','Enclosed Balcony','Owned','Agincourt North'],inplace = True)

In [40]:
tar = pd.read_csv('output.csv',header=0,index_col=0)

In [41]:
# use 1000 dollor as an unit
tar/=1000

In [42]:
df['Bed'].fillna(0,inplace=True)

In [43]:
# set the train set, valadition set and test set
train=df[:2500]
test=df[2500:]
train_tar=tar[:2500]
test_tar=tar[2500:]

In [7]:
# After the normalization, there will be some errors, so i did not implement the normalization. I will fix this in the future
'''
mean = train.mean(axis=0)
train -= mean
std = train.std(axis=0)
train /= std

test -= mean
test /= std
'''

In [44]:
# we used sklearn first and then use sm
from sklearn import linear_model
from sklearn import preprocessing
import statsmodels.api as sm
train=np.array(train)
train_tar=np.array(train_tar)
train_nonan=[]
train_tar_nonan=[]

for i in range(len(train)):
    if np.isnan(train[i]).any():
        continue
    else:
        train_nonan.append(train[i])
        train_tar_nonan.append(train_tar[i])
train_nonan=np.array(train_nonan)
train_tar_nonan=np.array(train_tar_nonan)
print(train_nonan.shape)
print(train_tar_nonan.shape)
#deleting the missing data 
# set the model
clf = linear_model.LinearRegression()
clf.fit(train_nonan,train_tar_nonan)

(2469, 155)
(2469, 1)


LinearRegression()

In [45]:
import statsmodels.api as sm
X = sm.add_constant(train_nonan)

# build up the model
model = sm.OLS(train_tar_nonan,X)

# fit the model
re=model.fit()

#print out the parameters and model summary
print(re.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.672
Model:                            OLS   Adj. R-squared:                  0.651
Method:                 Least Squares   F-statistic:                     31.66
Date:                Sun, 21 Nov 2021   Prob (F-statistic):               0.00
Time:                        23:10:38   Log-Likelihood:                -5025.4
No. Observations:                2469   AIC:                         1.035e+04
Df Residuals:                    2318   BIC:                         1.123e+04
Df Model:                         150                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          6.4445      0.618     10.421      0.0

In [49]:
# delete the missing data
test=np.array(test)
test_tar=np.array(test_tar)
test_nonan=[]
test_tar_nonan=[]
np.isnan(test[1])
for i in range(len(test)):
    if np.isnan(test[i]).any():
        continue
    else:
        test_nonan.append(test[i])
        test_tar_nonan.append(test_tar[i])
test_nonan_con = sm.add_constant(test_nonan)
pre=re.predict(test_nonan_con)

In [55]:
mae=0
mse=0
for i in range(len(pre)):
    mae+=abs(pre[i]-test_tar_nonan[i][0])
    mse+=(pre[i]-test_tar_nonan[i][0])**2
mae/=len(pre)
mse/=len(pre)
print('mae is: ',mae)
print('mse is: ',mse)
print('the number of test set is: ',len(pre))

mae is:  1.2176255928961273
mse is:  2.6454480871022463
the number of test set is:  402


## SVM regression
In this part, we are going to use svm regression to predict prices

In [56]:
from sklearn.svm import NuSVR

In [62]:
svmodel=NuSVR(C=1.0, nu=0.1)

array([[ 7.606],
       [ 9.273],
       [11.605],
       ...,
       [15.152],
       [ 9.677],
       [10.7  ]])

In [64]:
re=svmodel.fit(train_nonan,train_tar_nonan)

C:\Users\61910\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [68]:
pre2=re.predict(test_nonan)

In [72]:

mae2=0
mse2=0
for i in range(len(pre2)):
    mae2+=abs(pre2[i]-test_tar_nonan[i][0])
    mse2+=(pre2[i]-test_tar_nonan[i][0])**2
mae2/=len(pre2)
mse2/=len(pre2)
print('mae is: ',mae2)
print('mse is: ',mse2)
print('the number of test set is: ',len(pre2))

mae is:  1.9097470304781723
mse is:  5.782020305531429
the number of test set is:  402


The result of SVM regression is not idea, so we have to agjust the parameters. 

In [76]:
svm_re=[[] for i in range(5)]
C=[10**i for i in range(-4,4)]
gamma=[0.1,0.01,0.001]
nu_all=[0.1*i for i in range(1,10,2)]
co=0
for c in C:
    for g in gamma:
        for n in nu_all:
            print(co)
            co+=1
            svmodel=NuSVR(C=c, nu=n,gamma=g)
            re=svmodel.fit(train_nonan,train_tar_nonan)
            pre2=re.predict(test_nonan)
            mae2=0
            mse2=0
            for i in range(len(pre2)):
                mae2+=abs(pre2[i]-test_tar_nonan[i][0])
                mse2+=(pre2[i]-test_tar_nonan[i][0])**2
            mae2/=len(pre2)
            mse2/=len(pre2)
            svm_re[0].append(mae2)
            svm_re[1].append(mse2)
            svm_re[2].append(c)
            svm_re[3].append(g)
            svm_re[4].append(n)
            
            
            
            

0
1


C:\Users\61910\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\61910\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


2


C:\Users\61910\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


3


...

C:\Users\61910\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


118


C:\Users\61910\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


119


C:\Users\61910\anaconda3\lib\site-packages\sklearn\utils\validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [77]:
# the best mse and mae
print(len(svm_re[0]))
print(min(svm_re[0]))
print(min(svm_re[1]))

120
1.5123635551074484
4.163147013347549
